In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-20m.zip

--2023-02-14 11:44:21--  https://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  28.5MB/s    in 7.4s    

2023-02-14 11:44:29 (25.8 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]



In [ ]:
!unzip -n ml-20m.zip

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


In [ ]:
df=pd.read_csv(r'/content/ml-20m/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [ ]:
df.userId.value_counts()

118205    9254
8405      7515
82418     5646
121535    5520
125794    5491
          ... 
89305       20
110463      20
96990       20
134747      20
6526        20
Name: userId, Length: 138493, dtype: int64

In [ ]:
df.movieId=pd.Categorical(df.movieId)
df['movieId_new']=df.movieId.cat.codes


In [ ]:
df.userId=pd.Categorical(df.userId)
df['userId_new']=df.userId.cat.codes

In [ ]:
df.head()

,userId,movieId,rating,timestamp,movieId_new,userId_new
0,1,2,3.5,1112486027,1,0
1,1,29,3.5,1112484676,28,0
2,1,32,3.5,1112484819,31,0
3,1,47,3.5,1112484727,46,0
4,1,50,3.5,1112484580,49,0


In [ ]:
movie_id=df.movieId_new.values
user_id=df.userId_new.values
rating=df.rating.values
N=len(set(movie_id))
M=len(set(user_id))

In [ ]:
u = tf.keras.layers.Input(shape=(1,))
m = tf.keras.layers.Input(shape=(1,))

D=10 # optional dimention for Embedding

emb_i = tf.keras.layers.Embedding (N, D) (u) # output is T x D 
emb_m = tf.keras.layers.Embedding (M, D) (m) # output is T x D 

emb_i = tf.keras.layers.Flatten()(emb_i) 
emb_m = tf.keras.layers.Flatten()(emb_m) 


x = tf.keras.layers.Concatenate()([emb_i,emb_m] )# output is T* x M 




x = tf.keras.layers.Dense(1024, activation="relu")(x) # output is 1 
x = tf.keras.layers.Dense(1)(x) # output is 1 
model=tf.keras.Model(inputs=[u,m],outputs=x)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 10)        267440      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 10)        1384930     ['input_2[0][0]']                
                                                                                              

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.08,momentum=0.9),loss='mse')

In [ ]:
from sklearn.utils import shuffle
movie_id,user_id,rating=shuffle(movie_id,user_id,rating)
N_train=int(0.8*len(rating))

movie_id_train=movie_id[:N_train]
user_id_train=user_id[:N_train]
rating_train=rating[:N_train]



movie_id_test=movie_id[N_train:]
user_id_test=user_id[N_train:]
rating_test=rating[N_train:]

In [ ]:
rating_test

array([3. , 3. , 3.5, ..., 2.5, 5. , 4.5])

In [ ]:
avg_rating=rating_train.mean()
rating_train=rating_train-avg_rating
rating_test=rating_test-avg_rating

In [ ]:
r=model.fit(x=[movie_id_train,user_id_train],
            y=rating_train,validation_data=([movie_id_test,user_id_test],rating_test)
            ,epochs=25,batch_size=1024,verbose=2)

Epoch 1/25
15626/15626 - 78s - loss: 0.7666 - val_loss: 0.7157 - 78s/epoch - 5ms/step
Epoch 2/25
15626/15626 - 64s - loss: 0.6989 - val_loss: 0.6938 - 64s/epoch - 4ms/step
Epoch 3/25
15626/15626 - 62s - loss: 0.6760 - val_loss: 0.6805 - 62s/epoch - 4ms/step
Epoch 4/25
15626/15626 - 61s - loss: 0.6622 - val_loss: 0.6723 - 61s/epoch - 4ms/step
Epoch 5/25
15626/15626 - 65s - loss: 0.6499 - val_loss: 0.6670 - 65s/epoch - 4ms/step
Epoch 6/25


KeyboardInterrupt: ignored